Installing keras tuner

In [2]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 133 kB 34.7 MB/s 


Importing libraries

In [3]:
import tensorflow as tf 
from tensorflow import keras
import numpy as np

Checking tensorflow version

In [4]:
print(tf.__version__)

2.8.2


In [5]:
fashion_mnist = keras.datasets.fashion_mnist

Loading data and splitting train test images and labels

In [6]:
(trainimages, trainlabels), (testimages, testlabels) = fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


Standard scaling the images

In [7]:
trainimages = trainimages/255.0
testimages = testimages/255.0

Checking the shape of the images

In [8]:
trainimages[0].shape

(28, 28)

Reshaping the images

In [9]:
trainimages = trainimages.reshape(len(trainimages), 28, 28, 1)
testimages = testimages.reshape(len(testimages), 28, 28, 1)

Building a model for CNN and selecting the best parameters 

In [10]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16), 
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=tf.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

Importing libraries

In [11]:
import kerastuner
import keras
import kerastuner.engine.hyperparameters as hp_module
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


Random Search 

In [12]:
tunersearch = RandomSearch(build_model, objective='val_accuracy', max_trials=5, 
                           directory='/content/drive/MyDrive/Deep Learning', 
                           project_name='fashion')

INFO:tensorflow:Reloading Oracle from existing project /content/drive/MyDrive/Deep Learning/fashion/oracle.json


Searching for the best parameters

In [13]:
tunersearch.search(trainimages, trainlabels, epochs=4, validation_split=0.1)

Trial 5 Complete [00h 00m 31s]
val_accuracy: 0.9088333249092102

Best val_accuracy So Far: 0.9133333563804626
Total elapsed time: 00h 02m 58s
INFO:tensorflow:Oracle triggered exit


Getting the best model

In [16]:
model = tunersearch.get_best_models(num_models=1)[0]

Model Summary to check what parameters were selected

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 48)        1248      
                                                                 
 conv2d_1 (Conv2D)           (None, 20, 20, 48)        57648     
                                                                 
 flatten (Flatten)           (None, 19200)             0         
                                                                 
 dense (Dense)               (None, 64)                1228864   
                                                                 
 dense_1 (Dense)             (None, 10)                650       
                                                                 
Total params: 1,288,410
Trainable params: 1,288,410
Non-trainable params: 0
_________________________________________________________________


Fit the model now

In [18]:
model.fit(trainimages, trainlabels, epochs=10, validation_split=0.1, initial_epoch=4)

Epoch 5/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1403 - accuracy: 0.9476 - val_loss: 0.2715 - val_accuracy: 0.9082
Epoch 6/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1080 - accuracy: 0.9583 - val_loss: 0.2786 - val_accuracy: 0.9132
Epoch 7/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0883 - accuracy: 0.9662 - val_loss: 0.3241 - val_accuracy: 0.9090
Epoch 8/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0684 - accuracy: 0.9740 - val_loss: 0.3818 - val_accuracy: 0.9057
Epoch 9/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0543 - accuracy: 0.9801 - val_loss: 0.4226 - val_accuracy: 0.9068
Epoch 10/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0488 - accuracy: 0.9821 - val_loss: 0.4945 - val_accuracy: 0.9005


Validation accuracy and train accuracy both are good!